# Extract Data From Api

In [10]:
import requests
import pandas as pd
import time

# 1. Access Configurations
# REPLACE 'your_api_token_here' with your actual API token from football-data.org
API_TOKEN = 'your_api_token_here'
HEADERS = { 'X-Auth-Token': API_TOKEN }

# 2. Target Leagues Selection (Big Five + Champions League)
LEAGUES = {
    'PL': 'Premier League',
    'BL1': 'Bundesliga',
    'PD': 'La Liga',
    'FL1': 'Ligue 1',
    'CL': 'Champions League'
}

# 3. Timeframe Definition (Historical Data for the last 3 seasons)
SEASONS = [2023, 2024, 2025]
all_match_data = []

print("Starting Data Extraction: Fetching leagues details...")

for code, name in LEAGUES.items():
    for season in SEASONS:
        # Construct the API endpoint for each league and season
        url = f"https://api.football-data.org/v4/competitions/{code}/matches?season={season}"

        try:
            response = requests.get(url, headers=HEADERS)

            if response.status_code == 200:
                data = response.json()
                matches = data.get('matches', [])

                # Iterate through each match to extract specific features
                for m in matches:
                    all_match_data.append({
                        'League': name,
                        'Season': season,
                        'Date': m.get('utcDate'),
                        'Stage': m.get('stage'),             # Type of match (e.g., FINAL, GROUP_STAGE)
                        'Home_Team': m.get('homeTeam', {}).get('name'),
                        'Away_Team': m.get('awayTeam', {}).get('name'),
                        'Home_Logo': m.get('homeTeam', {}).get('crest'), # Team Logos for Power BI
                        'Away_Logo': m.get('awayTeam', {}).get('crest'),
                        'Home_Goals': m.get('score', {}).get('fullTime', {}).get('home'),
                        'Away_Goals': m.get('score', {}).get('fullTime', {}).get('away'),
                        'Winner': m.get('score', {}).get('winner')
                    })

                # Sleep for 6 seconds to comply with the free tier rate limit (10 requests/min)
                time.sleep(6)
                print(f" Successfully fetched: {name} - Season {season}")
            else:
                print(f" Failed to fetch {name}: Status Code {response.status_code}")

        except Exception as e:
            print(f" An unexpected error occurred: {e}")

# 4. Data Processing and Export
df_final = pd.DataFrame(all_match_data)

# Convert Date column to a cleaner Date format
df_final['Date'] = pd.to_datetime(df_final['Date']).dt.date

# Export the final dataset to Excel for Power BI analysis
output_file = "Leagues_Historical_Data.xlsx"
df_final.to_excel(output_file, index=False)

print("\n Process Completed!")
print(f"Dataset saved as: {output_file}")
print(f"Total Matches Extracted: {len(df_final)}")

Starting Data Extraction: Fetching leagues details...
 Successfully fetched: Premier League - Season 2023
 Successfully fetched: Premier League - Season 2024
 Successfully fetched: Premier League - Season 2025
 Successfully fetched: Bundesliga - Season 2023
 Successfully fetched: Bundesliga - Season 2024
 Successfully fetched: Bundesliga - Season 2025
 Successfully fetched: La Liga - Season 2023
 Successfully fetched: La Liga - Season 2024
 Successfully fetched: La Liga - Season 2025
 Successfully fetched: Ligue 1 - Season 2023
 Successfully fetched: Ligue 1 - Season 2024
 Successfully fetched: Ligue 1 - Season 2025
 Successfully fetched: Champions League - Season 2023
 Successfully fetched: Champions League - Season 2024
 Successfully fetched: Champions League - Season 2025

 Process Completed!
Dataset saved as: Leagues_Historical_Data.xlsx
Total Matches Extracted: 4619


In [11]:

print(df_final.head())

           League  Season        Date           Stage  \
0  Premier League    2023  2023-08-11  REGULAR_SEASON   
1  Premier League    2023  2023-08-12  REGULAR_SEASON   
2  Premier League    2023  2023-08-12  REGULAR_SEASON   
3  Premier League    2023  2023-08-12  REGULAR_SEASON   
4  Premier League    2023  2023-08-12  REGULAR_SEASON   

                   Home_Team             Away_Team  \
0                 Burnley FC    Manchester City FC   
1                 Arsenal FC  Nottingham Forest FC   
2            AFC Bournemouth    West Ham United FC   
3  Brighton & Hove Albion FC         Luton Town FC   
4                 Everton FC             Fulham FC   

                                          Home_Logo  \
0          https://crests.football-data.org/328.png   
1           https://crests.football-data.org/57.png   
2  https://crests.football-data.org/bournemouth.png   
3          https://crests.football-data.org/397.png   
4           https://crests.football-data.org/62.png   

 

In [12]:
print(df_final.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4619 entries, 0 to 4618
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   League      4619 non-null   object 
 1   Season      4619 non-null   int64  
 2   Date        4619 non-null   object 
 3   Stage       4619 non-null   object 
 4   Home_Team   4574 non-null   object 
 5   Away_Team   4574 non-null   object 
 6   Home_Logo   4574 non-null   object 
 7   Away_Logo   4574 non-null   object 
 8   Home_Goals  3902 non-null   float64
 9   Away_Goals  3902 non-null   float64
 10  Winner      3902 non-null   object 
dtypes: float64(2), int64(1), object(8)
memory usage: 397.1+ KB
None


In [13]:

print(df_final.columns)

Index(['League', 'Season', 'Date', 'Stage', 'Home_Team', 'Away_Team',
       'Home_Logo', 'Away_Logo', 'Home_Goals', 'Away_Goals', 'Winner'],
      dtype='object')


In [14]:

file_name = "Big_Five_Leagues_2023_2025.xlsx"
df_final.to_excel(file_name, index=False)

print(f"save {file_name}")

save Big_Five_Leagues_2023_2025.xlsx


In [15]:
from google.colab import files
files.download('Big_Five_Leagues_2023_2025.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>